# Carbon Seasonal Cycle Tutorial
The purpose of this tutorial is to demonstrate three methods of calculating the carbon seasonal cycle using atmospheric carbon dioxide observations. You will compare your results against known results for Park Falls, Wisconsin to confirm that you have implemented each method correctly. All three methods are commonly used, so it is up to you to choose the best method for your project.

## Option 1: Rolling average
When I was first tasked with creating a seasonal cycle, I did a Google search and came across this tutorial: https://datalab.marine.rutgers.edu/2020/03/modeling-seasonal-data/. The purpose is to use simple data analysis techniques to model a seasonal dataset. The notebook cells below will lead you through the creation of a test dataset that is comprised of a sinusoidal seasonal cycle, an increasing long-term trend, and random noise. This is a decent representation of atmospheric carbon dioxide (CO$_2$) observations. This dataset will then be used to show how you can model the long-term trend and seasonal cycle.
Once you feel comfortable doing this with the test data, you will implement the same method using real CO$_2$ observations for a carbon observing site in Park Falls, Wisconsin.

### Setup Notebook

In [ ]:
# Import necessary libraries and packages

# Data Manipulation
import numpy as np
import pandas as pd
import xarray as xr

# Data Visualization
import matplotlib.pyplot as plt

### Creating a Test Dataset

In [ ]:
# Initialize a 5-year timeseries dataset
df = pd.DataFrame(index=pd.date_range('2020-01-01','2025-01-01'))
df

In [ ]:
# Generate an annual seasonal cycle using a cosine function
mean = 5
offset = 10
df['y1'] = np.cos(df.index.dayofyear/365*2*np.pi - np.pi)*mean + offset
df

In [ ]:
# Plot data to check that it looks as expected
df['y1'].plot()
plt.grid()
plt.show()

In [ ]:
# Add a trend to the data
trend = 5
df['y2'] = df['y1'] + trend*np.arange(0,df.shape[0])/df.shape[0]
df

In [ ]:
# Plot data to check that it looks as expected
df.plot()
plt.grid()
plt.show()

In [ ]:
# Add random noise to the data
noise_mean = 0
noise_var = 2
df['y3'] = df['y2'] + np.random.normal(noise_mean, noise_var, df.shape[0])
df

In [ ]:
# Plot data to check that it looks as expected
df['y3'].plot(marker='.', markersize=2, linestyle='')
plt.title('Test Seasonal Dataset with Trend')
plt.grid()
plt.show()

### Detrending Data
The seasonal dataset you have created is now a decent representation of what atmospheric CO$_2$ observations look like. We will not reverse the process to extract the long-term trend and seasonal cycle.
The first step of getting a seasonal cycle will often be to detrend your dataset. This models, then removes the long-term trend, making it easier to see and analyze changes that occur on shorter timescales. On short timescales (about 2 decades or less) the increase in atmospheric CO$_2$ will be roughly linear. Therefore, fitting a linear model of the form

$y = mx + b$

to the data will model the long-term trend, which can then be removed.
There are many ways to do this, but we will use the numpy.polyfit() function. The function will fit a polynomial or given order to the input dataset and returns the coefficient. The first element is the slope and the second is the y-intercept.

In [ ]:
# Fit a linear trend
df2 = df.reset_index() # This process doesn't work for time indexes, so we need to reset
coeff = np.polyfit(df2.index,df2.y3,deg=1)

print('Slope (m): %f' % coeff[0])
print('Intercept (b): %f' % coeff[1])

In [ ]:
# Check that the result makes sense by calculating annual increase
# The trend we added increased by 5 over 5 years, or about 1 per year, so the result should be about 1
print('Annual Increase: %f' % (coeff[0]*365))

# Add the modeled trend to our dataset
model = np.poly1d(coeff)
df['trend'] = model(df2.index)
df

In [ ]:
# Plot data to check that it looks as expected
df['y3'].plot(marker='.', markersize='2', linestyle='', label='Raw Data')
df['trend'].plot(label='Linear Trend')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Remove the trend
df['deseasonalized'] = df['y3'] - df['trend']
df

In [ ]:
# Plot data to check that it looks as expected
df['deseasonalized'].plot(marker='.', markersize='2', linestyle='')
plt.grid()
plt.show()

### Modeling the Seasonal Cycle
There are many ways to model periodic processes like the seasonal cycle. Here, we will assume that we know nothing about the process leading to seasonality, only that it repeats annually. We will model the seasonality by calculating the expected value for each day of the year.

In [ ]:
# Add a day of the year column to dataset
df['yearday'] = df.index.dayofyear
df

In [ ]:
# Plot data to check that it looks as expected
plt.plot(df.yearday, df.deseasonalized, marker='.', markersize=2, linestyle='')
plt.xlabel('Day of Year')
plt.grid()
plt.show()

In [ ]:
# Calculate seasonal cycles
yr = df.deseasonalized.groupby(df.yearday).mean()
yr.plot(label='1-day average')

yr7 = yr.rolling(7, center=True, min_periods=4).mean()
yr7.plot(label='7-day average')

yr30 = yr.rolling(30, center=True, min_periods=15).mean()
yr30.plot(label='30-day average')
plt.legend()
plt.xlabel('Day of Year')
plt.grid()
plt.show()

In [ ]:
# Create seasonal model
df['seasonal'] = yr30[df.yearday].values
df

In [ ]:
# Plot data to check that it looks as expected
df['seasonal'].plot()
plt.grid()
plt.show()

In [ ]:
# Lastly, let's compare our model to our original dataset
df['y3'].plot(label='Dataset', marker='.', markersize=2, linestyle='')
(df['trend'] + df['seasonal']).plot(label='Model')
plt.title('Measured and Modeled Seasonal Datasets')
plt.legend()
plt.grid()
plt.show()

### Repeat Using Real Observations
Import data from Park Falls, Wisconsin and find the seasonal cycle. Most atmospheric CO$_2$ datasets are in the netCDF file format. This means that using xarray, rather than pandas can be beneficial when working with these datasets. You have the choice of converting your xarray DataSet into a pandas DataFrame and completing the tutorial as above, or making some slight variations so that you can work with the DataSet directly.

In [ ]:
# Import Park Falls dataset
pa_ds = xr.open_dataset('pa_co2.nc')
pa_ds

In [ ]:
# Select only carbon dioxide data from full dataset
pa_ds = pa_ds['xco2'].to_dataset(name='xco2')
pa_ds

In [ ]:
# If desired, convert xarray DataArray to pandas DataFrame
# The remaining tutorial will work with xarray, not pandas
# Repeat the process above if you want to continue working with pandas instead
pa_df = pa_ds.to_dataframe()
pa_df

In [ ]:
# Plot raw data
pa_ds.xco2.plot(marker='.', markersize=2, linestyle='')
plt.grid()
plt.show()

In [ ]:
# Fit a linear trend
coeff = pa_ds.xco2.polyfit(dim='time',deg=1)

print('Slope (m): %f' % coeff.polyfit_coefficients[0])
print('Offset (b): %f' % coeff.polyfit_coefficients[1])

# Model the trend
model = xr.polyval(coord=pa_ds.time, coeffs=coeff.polyfit_coefficients)
pa_ds['trend'] = model
pa_ds

In [ ]:
# Plot raw data and modeled linear trend 
pa_ds.xco2.plot(marker='.', markersize=2, linestyle='', label='Raw Data')
pa_ds.trend.plot(label='Linear Trend')
plt.legend()
plt.grid()
plt.show()

# Remove linear trend
pa_ds['deseasonalized'] = pa_ds['xco2'] - model

# Plot deseasonalized data
pa_ds.deseasonalized.plot(marker='.', markersize=2, linestyle='')
plt.grid()
plt.show()

In [ ]:
# Model the seasonal cycle
pa_yr = pa_ds.deseasonalized.groupby(pa_ds.time.dt.dayofyear).mean()
pa_yr.plot(label='1-day average')

pa_yr7 = pa_yr.rolling(dayofyear=7, center=True, min_periods=4).mean()
pa_yr7.plot(label='7-day average')

pa_yr30 = pa_yr.rolling(dayofyear=30, center=True, min_periods=15).mean()
pa_yr30.plot(label='30-day average')
plt.legend()
plt.grid()
plt.show()

# Add seasonal model to dataset
doy = pa_ds.time.dt.dayofyear.to_series()
yr30_s = pa_yr30.to_series()
pa_ds['seasonal'] = ('time',yr30_s[doy].values)
pa_ds

In [ ]:
# Plot seasonal data
pa_ds.seasonal.plot(marker='.', markersize=2, linestyle='')
plt.grid()
plt.show()

In [ ]:
# Compare model to original dataset
pa_ds['xco2'].plot(label='Dataset', marker='.', markersize=2, linestyle='')
(pa_ds['trend'] + pa_ds['seasonal']).plot(label='Model')
plt.title('Measured and Modeled Seasonal Datasets')
plt.legend()
plt.grid()
plt.show()

## Option 2: STL
I next moved to using STL to decompose data into a long-term trend and seasonal cycle. STL, or Seasonal-trend Decomposition using LOESS (locally estimated scatterplot smoothing) is a statistical method described at http://bit.ly/stl1990. STL provides a lot of benefit over other methods in that it can handle any time of seasonality, the seasonal component can change over time, the smoothness of the trend can be controlled by the user, and it is robust to outliers. However, it can only handle additive decompositions and can be a mystery if you don't understand the math because you will use a function rather than finding the trend and seasonal cycle yourself. We will use the STL function found in the statsmodels package

### Setup Notebook
Most of the necessary imports were done above, so we will only add new things here

In [ ]:
from statsmodels.tsa.seasonal import STL

### Apply STL

In [ ]:
# Apply STL to test dataset
monthly = df['y3'].resample('ME').mean().ffill()
stl = STL(monthly, period=12, seasonal=13)
res = stl.fit()
fig = res.plot()

In [ ]:
# Compare raw data to model
df['y3'].plot(marker='.', markersize=2, linestyle='', label='Raw Data')
(res.trend + res.seasonal).plot(label='Model')
plt.legend()
plt.grid()
plt.show()

### Repeat with Park Falls

In [ ]:
# Apply STL to dataset
pa_ds = pa_ds.sortby('time')
pa_monthly = pa_ds.xco2.resample(time='ME').mean().ffill(dim='time').to_series()
pa_stl = STL(pa_monthly, period=12, seasonal=13, robust=True)
pa_res = pa_stl.fit()
pa_fig = pa_res.plot()

In [ ]:
# Compare raw data to model
pa_ds.xco2.plot(marker='.', markersize=2, linestyle='', label='Raw Data')
(pa_res.trend + pa_res.seasonal).plot(label='Model')
plt.legend()
plt.grid()
plt.show()

## Option 3: Hamonic Fit
Most recently, I've moved to fitting a harmonic to the dataset, removing the long-term trend, and finding the seasonal cycle of the fit. In the previous two methods, we assumed that there was some periodicity to the data, but we didn't know the processes that led to that periodicity. However, we do know that the periodic nature of the data is largely related to photosynthesis and respiration and that the seasonal cycle can almost always be well fit by a skew harmonic function, so it is beneficial to use that function to start with.

### Setup Notebook
Import libraries and packages needed here that have not been previously imported

In [ ]:
import matplotlib.dates as mdates
from scipy.optimize import leastsq

### Define Functions for Fitting

In [ ]:
# Define function to fit a harmonic to the data
def fit_harm(data): 
    p0 = [250, 0.006, -4, -111, 0.7, -290]
    monthvals = mdates.date2num(data['time']) - 1970 * 365
    errfunc = lambda p, t, y: fit_function(p, t) - y
    p2, success = leastsq(errfunc, p0[:], args=(monthvals, data['xco2']))
    if success != 1 & success != 2 & success != 3 & success != 4:
        print('The fit was unsuccessful')
        print(success)
    return p2

# Function that is fit to the data
fit_function = lambda p,t: p[0] + p[1]*t + p[2]*np.sin((2*np.pi/365)*(t-p[3])+np.arccos(p[4]*np.cos((2*np.pi/365)*t-p[5])))
p0 = [ 2.85711733e+02,  6.77853118e-03,  3.59468633e+00, -1.75291954e+02, -5.50459245e-01, -2.94022555e+02]

### Apply Functions to Data

In [ ]:
# Reset index and rename column to work with function
df = df.reset_index()
df = df.rename(columns={'index':'time','y3':'xco2'})
df

In [ ]:
# Apply function to get coefficients that result in the best fit
p2 = fit_harm(df)
p2

In [ ]:
# Create model using coefficients
date_range = pd.date_range(start=df['time'].iloc[0], end=df['time'].iloc[-1], freq='D')
fit = fit_function(p2, mdates.date2num(date_range) - 1970 * 365)

# Plot model and raw data to compare
plt.plot(df.time, df.xco2, marker='.', markersize=2, linestyle='', label='Raw Data')
plt.plot(date_range, fit, marker='.', markersize=2, linestyle='', label='Model')
plt.legend()
plt.grid()
plt.show()

### Repeat for Park Falls

In [ ]:
# Apply function to get coefficients that result in the best fit
pa_p2 = fit_harm(pa_ds)
pa_p2

In [ ]:
# Create model using coefficients
pa_date_range = pd.date_range(start=pa_ds['time'][0].item(), end=pa_ds['time'][-1].item(), freq='D')
pa_fit = fit_function(pa_p2, mdates.date2num(pa_date_range) - 1970 * 365)

# Plot model and raw data to compare
plt.plot(pa_ds.time, pa_ds.xco2, marker='.', markersize=2, linestyle='', label='Raw Data')
plt.plot(pa_date_range, pa_fit, marker='.', markersize=2, linestyle='', label='Model')
plt.legend()
plt.grid()
plt.show()

At this point, you can apply any detrending method you like to find the seasonal cycle. In addition to the methods share above, I've also used Mauna Loa data to detrend. The CO$_2$ observing station at Mauna Loa, Hawaii, is one of the longest-running and is thought to be representative of the entire Northern Hemisphere. In my work, I have always either fit a line to the Mauna Loa data or applied STL to the Mauna Loa data in order to get the trend that is then removed from the dataset. This means that I always remove the same increasing trend, regardless of dataset. I've chosen not to include Mauna Loa detrending in this tutorial because the Mauna Loa dataset required a lot of processing before it can be compatible with the datasets we will be using. If you have extra time or are just intrested in using Mauna Loa data to detrend, please reach out so that I can share some functions I have written that detrends using Mauna Loa.

## Putting it all Together
Let's now compare all three options to see which model we like the best. Plot all models against the raw data on the same plot.

In [ ]:
# Plot test data
plt.plot(df.time, df.xco2, marker='.', markersize=2, linestyle='', label='Raw Data')
plt.plot(df.time, (df.trend + df.seasonal), label='Rolling Average')
plt.plot(res.trend.index, (res.trend + res.seasonal), label='STL')
plt.plot(date_range, fit, label='Harmonic Fit')
plt.legend()
plt.title('Test Dataset')
plt.grid()
plt.show()

In [ ]:
# Plot Park Falls data
pa_ds.xco2.plot(marker='.', markersize=2, linestyle='', label='Raw Data')
(pa_ds['trend'] + pa_ds['seasonal']).plot(label='Rolling Average')
plt.plot(pa_res.trend.index, (pa_res.trend + pa_res.seasonal), label='STL')
plt.plot(pa_date_range, pa_fit, label='Harmonic Fit')
plt.legend()
plt.title('Park Falls')
plt.grid()
plt.show()

Check that your result matches mine!


<img src="ParkFallsSeasonal.png">

Now, let's look at the mean seasonal cycle and the seasonal cycle amplitude (SCA) which is typically defined to be the maximum value minus the minimum value of the seasonal cycle. You should get an SCA value for Park Falls of about 9.07 ppm when taking a rolling average, 7.15 ppm when using STL, and 9.23 ppm when fitting a harmonic. In my experience, when I've used STL, I always have to fit a harmonic to the seasonal cycle in order to get anything useful. If you decide to move forward with using STL, you should plan to fit a harmonic to the seasonal cycle.

In [ ]:
# Create dataframe holding detrended harmonic fit data to make it easier to find the mean seasonal cycle
pa_fit_detrended = pa_fit - (pa_p2[0] + pa_p2[1]*(mdates.date2num(pa_date_range) - 1970 * 365))
pa_fit_df = pd.DataFrame(pa_fit_detrended, columns=['xco2'])
pa_fit_df.set_index(pa_date_range, inplace=True)

In [ ]:
# Find the mean seasonal cycle for each method
rolling_sc = pa_yr30
stl_sc = pa_res.seasonal.groupby(pa_res.seasonal.index.dayofyear).mean()
harmonic_sc = pa_fit_df.groupby(pa_fit_df.index.dayofyear).mean()

In [ ]:
# Plot mean seasonal cycle for all three methods
plt.plot(rolling_sc.dayofyear, rolling_sc, label='Rolling Average')
plt.plot(stl_sc.index, stl_sc, label='STL')
plt.plot(harmonic_sc.index, harmonic_sc, label='Harmonic Fit')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Find and compare the seasonal cycle amplitudes
print('Rolling Average SCA: %f ppm' % (rolling_sc.max() - rolling_sc.min()))
print('STL SCA: %f ppm' % (stl_sc.max() - stl_sc.min()))
print('Harmonic Fit SCA: %f ppm' % (harmonic_sc.max() - harmonic_sc.min()).values.item())